In [17]:
#2019.10.18 准备写一个大盘分析程序，将相关数据显示在网页上，然后后台定时运行，分析每天结果，我想看时打开网页就行。
#2019.10.24 本段程序实现往前算指定天数ndays，将index_dailybasic中的总市值与每日指数中成交量、成交额合并一个DF显示
import tushare as ts
from datetime import date, datetime, timedelta
pro = ts.pro_api() 
#初始化
ndays=10
#ndays将来计划是由网页提交给数据库，可以由用户更改。
delta = timedelta(days=8)  
#timedelta可以实现日期时间直接相加，例子如下：
#now = datetime.now()
#day = date.today()
#n_days_after = now + delta  #当前日期推迟n天之后的时间
#n_days_forward  = now - delta  #当前日期向前推n天的时间
day=date.today()
#取今天的时间
end_date=(day-timedelta(days=1)).strftime('%Y%m%d')
#截止时间为前一天,#默认输出是2019-10-24的格式，用strftime换算成20191024
start_date=(day-timedelta(days=ndays)).strftime('%Y%m%d')
#开始时间根据ndays的不同，可以变化
df1 = pro.index_dailybasic(start_date=start_date,end_date=end_date,ts_code='000001.SH',fields='ts_code,trade_date,total_mv,turnover_rate')
#index_dailybasic提供大盘指数每日行情，目前主要是目前只提供上证综指，深证成指，上证50
#中证500，中小板指，创业板指的每日指标数据
df2 = pro.index_daily(ts_code='000001.SH',start_date=start_date,end_date=end_date,fields='ts_code,trade_date,close,change,pct_chg,vol,amount')
#指数每日行情，用主要输出参数close,open,high,low,pre_close
#change涨跌点相对昨天收价，pct_chg涨跌幅相对昨天收盘价
#vol成交量手，amount成交额千元
df2['total_mv']=df1.total_mv
#将df1的total_mv合并df2
#dataframe增加一列用法，df['new']=[1,2,3,4] 
#dataframe增加一行用法，加loc,df.loc['new']=[1,2,3,4]
#--------------------我是分割线------------------------------------------------------------
#2019.10.19
#原本想将运行结果保存到数据库，然后用PHP读出来显示到网页上，今天突然想到PYTHON能不能直接保存结果为HTML？百度一下，居然发现DATAFRAME可以
#直接存为HTML文件，这下省事了。
df2.to_html('files.html',escape=False,index=False,sparsify=True,border=1,index_names=True,header=True)

df2

,ts_code,trade_date,close,change,pct_chg,vol,amount,total_mv
0,000001.SH,20191023,2941.6188,-12.7611,-0.4319,123812881.0,139653650.5,3.862913e+13
1,000001.SH,20191022,2954.3799,14.7620,0.5022,117482048.0,130381741.2,3.877605e+13
2,000001.SH,20191021,2939.6179,1.4766,0.0503,132475107.0,145685881.2,3.860444e+13
3,000001.SH,20191018,2938.1413,-39.1929,-1.3164,149990678.0,162018272.3,3.852931e+13
4,000001.SH,20191017,2977.3342,-1.3782,-0.0463,122527614.0,136051252.1,3.902802e+13
5,000001.SH,20191016,2978.7124,-12.3335,-0.4123,149885496.0,166791549.4,3.904096e+13
6,000001.SH,20191015,2991.0459,-16.8375,-0.5598,155376904.0,169531932.5,3.920118e+13
7,000001.SH,20191014,3007.8834,34.2276,1.1510,208614802.0,221201755.7,3.939485e+13


In [7]:
#2019.10.18 准备写一个大盘分析程序，将相关数据显示在网页上，然后后台定时运行，分析每天结果，我想看时打开网页就行。
import tushare as ts
pro = ts.pro_api() 
#初始化
df1 = pro.index_dailybasic(start_date='20191008',end_date='20191022',ts_code='399001.SZ',fields='ts_code,trade_date,total_mv,turnover_rate')
#index_dailybasic提供大盘指数每日行情，目前主要是目前只提供上证综指，深证成指，上证50
#中证500，中小板指，创业板指的每日指标数据
df2 = pro.index_daily(ts_code='399001.SZ',start_date='20191008',end_date='20191022')
#指数每日行情，用主要输出参数close,open,high,low,pre_close
#change涨跌点相对昨天收价，pct_chg涨跌幅相对昨天收盘价
#vol成交量手，amount成交额千元
df2['total_mv']=df1.total_mv
#将df1的total_mv合并df2
#dataframe增加一列用法，df['new']=[1,2,3,4] 
#dataframe增加一行用法，加loc,df.loc['new']=[1,2,3,4]
#--------------------我是分割线------------------------------------------------------------
#2019.10.19
#原本想将运行结果保存到数据库，然后用PHP读出来显示到网页上，今天突然想到PYTHON能不能直接保存结果为HTML？百度一下，居然发现DATAFRAME可以
#直接存为HTML文件，这下省事了。
df2.to_html('files.html',escape=False,index=False,sparsify=True,border=1,index_names=True,header=True)



In [25]:
#本段采集深证官网首页大盘信息
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
opt = Options()
opt.add_argument('--headless')
# 创建chrome无界面对象
driver = webdriver.Chrome(options=opt)
#设置无界面
driver.get('http://www.szse.cn/')
#访问百度
a=driver.find_elements_by_class_name('tab-pane')
#大盘情况放在class为tab-pane名字中，find_elements找出所有的tab_pane
c=[]
for b in a:
    if b.text!='':
        #一共能找到四个class，其中三个为空，要过滤掉
        c=b.text.split('\n') 
        #以\n分割字符串
table_name=c[::2]
table_name.append('数据采集时间')
#增加列名：数据采集时间
table_value=c[1::2]

d=driver.find_elements_by_class_name('tip')
#时间放在tip中，首页中一共有两个tip，取最后一个
f=[]
for e in d:
    f.append(e.text)
table_value.append(f[-1])
#对应增加采集到的日期
df = pd.DataFrame([table_value], columns=table_name) 




In [ ]:
#多个DATAFRAME写入同一个html的例子
import os
HEADER ='''<html>
            <head>
                <meta charset="UTF-8">
            </head>
            <body>
            a1ddeeed'''
FOOTER = '''</body>
            </html>'''

 
with open(os.path.join(os.getcwd(), 'a.html'), 'w') as f:
    f.write(HEADER)
    f.write(FOOTER)
